In [23]:
import json
import pandas as pd
from tqdm import tqdm

In [38]:
def extract_data(data,df):
    ids = []
    adverse_reactions = []
    indications_and_usage = []
    contraindications = []
    warnings_and_cautions = []
    brand_names = []

    for drug in data['results']:
        try:
            ids.append(drug['id'])
        except:
            ids.append(None)

        try:
            brand_names.append(drug['openfda']['brand_name'][0])
        except:
            brand_names.append(None)

        try:
            adverse_reactions.append(drug['adverse_reactions'][0])
        except:
            adverse_reactions.append(None)

        try:
            indications_and_usage.append(drug['indications_and_usage'][0])
        except:
            indications_and_usage.append(None)
        
        try:
            contraindications.append(drug['contraindications'][0])
        except:
            contraindications.append(None)

        try:
            warnings_and_cautions.append(drug['warnings_and_cautions'][0])
        except:
            warnings_and_cautions.append(None)
    
    tdf = pd.DataFrame().assign(id=ids,brand_name=brand_names,adverse_reactions=adverse_reactions,indications_and_usage=indications_and_usage,
    contraindications=contraindications,warnings_and_cautions=warnings_and_cautions)

    df = pd.concat([df,tdf],axis=0).reset_index(drop=True)
    
    return(df)


### Extract Data
Using the openFDA json set, extract the relevant text fields from each application, where they exist. Save this data to a growing dataframe and then save the dataframe

In [39]:
file_numbers = ['0001','0002','0003','0004','0005','0006','0007','0008','0009','0010','0011']
fda_df = pd.DataFrame(columns=['id','adverse_reactions','indications_and_usage','contraindications','warnings_and_cautions','brand_names'])

for number in tqdm(file_numbers):
    f = open(f'../data/drug-label-{number}-of-0011.json','r')
    data = json.load(f)
    fda_df = extract_data(data,fda_df)



100%|██████████| 11/11 [00:41<00:00,  3.74s/it]


In [40]:
fda_df

,id,adverse_reactions,indications_and_usage,contraindications,warnings_and_cautions,brand_names,brand_name
0,b96b2e24-4192-31e2-e053-2a95a90a2356,ADVERSE REACTIONS SECTION The following are di...,INDICATIONS & USAGE SECTION To reduce the deve...,CONTRAINDICATIONS SECTION Amoxicillinfor oral ...,WARNINGS AND PRECAUTIONS SECTION 5.1 Anaphylac...,NaN,AMOXICILLIN AND CLAVULANATE POTASSIUM
1,c733ec78-5b43-7a7c-e053-2995a90a2f9a,None,Uses For the relief of symptoms associated wit...,None,None,NaN,UNDA 312
2,f21e2f96-6c05-fbe5-e053-2995a90a3948,None,Uses Helps prevent sunburn. If used as directe...,None,None,NaN,SUN PROTECT LIP BALM SPF 30
3,de170f72-0f59-4ed3-9715-8bdff49fc5da,None,None,None,None,NaN,LOSARTAN POTASSIUM AND HYDROCHLOROTHIAZIDE
4,e95e8654-1edf-40f7-8f42-38aacf3dcd08,6 ADVERSE REACTIONS The following clinically s...,1 INDICATIONS AND USAGE Potassium Phosphates I...,4 CONTRAINDICATIONS Potassium Phosphates Injec...,5 WARNINGS AND PRECAUTIONS Serious Cardiac Adv...,NaN,Potassium Phosphates
...,...,...,...,...,...,...,...
215910,2226fb84-50af-45b1-8a9f-72984e331519,ADVERSE REACTIONS The most common adverse reac...,INDICATIONS AND USAGE Carbidopa and levodopa t...,CONTRAINDICATIONS Nonselective monoamine oxida...,None,NaN,Carbidopa and Levodopa
215911,0776c9e3-a64d-40b2-b22d-aa66a5d4fa76,None,"Uses For the temporary relief of burning, irri...",None,None,NaN,REFRESH Optive Mega-3
215912,4ca80987-5d55-4476-a694-c95d3e07cf4c,6 ADVERSE REACTIONS The most serious adverse r...,1 INDICATIONS AND USAGE CREON ® is indicated f...,4 CONTRAINDICATIONS None. None ( 4 ),5 WARNINGS AND PRECAUTIONS Fibrosing colonopat...,NaN,Creon
215913,a725ae2b-865e-434c-9281-51f6344b97b9,6 ADVERSE REACTIONS Most common adverse reacti...,1 INDICATIONS AND USAGE Losartan potassium and...,4 CONTRAINDICATIONS Losartan potassium and hyd...,5 WARNINGS AND PRECAUTIONS Hypotension: Correc...,NaN,Losartan Potassium and Hydrochlorothiazide


In [41]:
fda_df.to_excel('../data/openfda.xlsx',sheet_name='09252023')

#### Scratch

In [6]:
data['results'][0].keys()

# additional fields of interest potentially: nonclinical_toxicology, clinical pharmacology?

dict_keys(['spl_product_data_elements', 'indications_and_usage', 'dosage_and_administration', 'dosage_and_administration_table', 'dosage_forms_and_strengths', 'contraindications', 'warnings_and_cautions', 'adverse_reactions', 'drug_interactions', 'use_in_specific_populations', 'overdosage', 'description', 'clinical_pharmacology', 'clinical_pharmacology_table', 'nonclinical_toxicology', 'clinical_studies', 'references', 'how_supplied', 'information_for_patients', 'package_label_principal_display_panel', 'set_id', 'id', 'effective_time', 'version', 'openfda'])

In [12]:
data['results'][0]['openfda'] # brand name, rxcui

{'application_number': ['ANDA065117'],
 'brand_name': ['AMOXICILLIN AND CLAVULANATE POTASSIUM'],
 'generic_name': ['AMOXICILLIN AND CLAVULANATE POTASSIUM'],
 'manufacturer_name': ['DIRECT RX'],
 'product_ndc': ['61919-019', '61919-401'],
 'product_type': ['HUMAN PRESCRIPTION DRUG'],
 'route': ['ORAL'],
 'substance_name': ['AMOXICILLIN', 'CLAVULANATE POTASSIUM'],
 'rxcui': ['308189', '617296'],
 'spl_id': ['b96b2e24-4192-31e2-e053-2a95a90a2356'],
 'spl_set_id': ['0173e9de-a995-4386-bb65-8fc2bbf347f9'],
 'package_ndc': ['61919-401-32', '61919-019-20'],
 'original_packager_product_ndc': ['65862-071', '66685-1002'],
 'unii': ['804826J2HU', 'Q42OMW3AT8']}